## 60대 데이터 늘리기

In [100]:
import torch
from timm.data.mixup import Mixup
from timm.data.dataset import ImageDataset
from timm.data.loader import create_loader
# import torchvision.transforms
import torchvision
import numpy as np
import os
from enum import Enum
from PIL import Image
import PIL

from torchvision.utils import save_image


In [101]:
data_directory = '/opt/ml/input/data/train/images'


In [107]:
def split_profile_by_gender_60s(profiles):
    male,female = [], []
    for profile in profiles:
        if profile.startswith("Fake"):  # 생성한 데이터를 제외한 데이터들을 반환하기 위해
            continue
        id, gender,species,age = profile.split("_")
        if age == "60":
            if gender == "male":
                male.append(os.path.join(data_directory,profile))
            else:
                female.append(os.path.join(data_directory,profile))
    return [male,female]

In [108]:
profiles = os.listdir(data_directory)
profiles = [profile for profile in profiles if not profile.startswith(".")]
male,female = split_profile_by_gender_60s(profiles)

In [111]:
import shutil
def make_images(profile_i,profile_j,save_dir,save_profile_dir,profiles):
    images = ["incorrect_mask","mask1","mask2","mask3","mask4","mask5","normal"]
    ext = ".jpg"
    # all
    for image in images:
        image_A = np.array(Image.open(os.path.join(profiles[profile_i], image+ext)))//2
        image_B = np.array(Image.open(os.path.join(profiles[profile_j], image+ext)))//2
        new_image = (image_A+image_B)
        img = PIL.Image.fromarray(new_image)
        img.save(os.path.join(save_dir,save_profile_dir,image+ext))    
    
def make_fakes_by_gender(gender,profiles,save_dir=data_directory):
    length = len(profiles)
    cnt = 0

    for folder in sorted(os.listdir(save_dir)):     # Fake 폴더 있는지 확인
        if folder.startswith("Fake"):
            cnt = int(folder.split("_")[1])
    for i in range(length//2):
        cnt += 1
        save_profile_dir = f"Fake_{cnt:0>6}_{gender}_Asian_60"
        os.mkdir(os.path.join(save_dir,save_profile_dir))
        j = i+length//2

        make_images(i,j,save_dir,save_profile_dir,profiles)
    # print(save_dir)
    
def make_fake_pics_not_random():
    make_fakes_by_gender("male",male)
    make_fakes_by_gender("female",female)
    print("Make Done.")
    
def rm_fake_pics(save_dir=data_directory):  # 만든 fake 디렉토리 전체 제거
    for fake in [i for i in os.listdir(save_dir) if i.startswith("Fake")]:
        fake_dir = os.path.join(save_dir,fake)
        shutil.rmtree(fake_dir)
    print("Remove Done.")



In [112]:
rm_fake_pics()
make_fake_pics_not_random()

Remove Done.
Make Done.
